## HW5 Мельчук А.Б.
Задание из 2-х частей.
Берем отызывы за лето (из архива с материалами или предыдущего занятия)


## 1. Учим conv сеть для классификации - выбить auc выше 0.95

In [1]:
import pandas as pd
import numpy as np

In [2]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
max_words = 2000
max_len = 40
num_classes = 1

# Training
epochs = 20
batch_size = 512
print_batch_n = 100

In [4]:
df = pd.read_excel("./отзывы за лето.xls")

In [5]:
df.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


### Предобработка

In [6]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [7]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in exclude]
    return " ".join(txt)

df['text'] = df['Content'].apply(preprocess_text)
df = df[df['Rating'] != 3]
df['target'] = df['Rating'] > 3

In [8]:
df['target'] = df['target'].astype(int)
df.head()

,Rating,Content,Date,text,target
0,5,It just works!,2017-08-14,it just works,1
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,в целое удобноной приложениеиз минус хотеть сл...,1
2,5,Отлично все,2017-08-14,отлично весь,1
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,стать зависать на 1 работа антивирус далёкий н...,1
4,5,"Очень удобно, работает быстро.",2017-08-14,очень удобно работать быстро,1


In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['target'], test_size=0.2, random_state=13, stratify=df['target'])

In [10]:
train_corpus = " ".join(X_train)
train_corpus = train_corpus.lower()

In [11]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

# from nltk.corpus import stopwords
# nltk.download('stopwords')
# stopwords = stopwords.words('english')
# !pip install stop_words
# nltk.download('all')

[nltk_data] Downloading package punkt to /home/ai/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [12]:
tokens = word_tokenize(train_corpus)

In [13]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [14]:
from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [15]:
tokens_filtered_top

['приложение',
 'весь',
 'и',
 'очень',
 'удобно',
 'в',
 'я',
 'на',
 'работать',
 'удобный',
 'с',
 'что',
 'отлично',
 'спасибо',
 'нравиться',
 'хороший',
 'отличный',
 'это',
 'хорошо',
 'телефон',
 'по',
 'супер',
 'после',
 'но',
 'раз',
 'мочь',
 'быть',
 'а',
 'за',
 'быстро',
 'просто',
 'обновление',
 'при',
 'пользоваться',
 'как',
 'так',
 'у',
 'антивирус',
 'устраивать',
 'вы',
 'сбербанк',
 'только',
 'не',
 'пароль',
 'пока',
 'карта',
 'то',
 'свой',
 'нормальный',
 'для',
 'прошивка',
 'вход',
 'без',
 'банк',
 'из',
 'рута',
 'перевод',
 'норма',
 'есть',
 'один',
 'сделать',
 'программа',
 'деньга',
 'разработчик',
 'счёт',
 'проблема',
 'писать',
 'время',
 'ошибка',
 'довольный',
 'бы',
 'ваш',
 'можно',
 'ок',
 'платёж',
 'или',
 'к',
 'мой',
 'он',
 'до',
 'стать',
 'другой',
 'постоянно',
 'приходиться',
 'через',
 'уже',
 'два',
 'понятно',
 'вводить',
 'если',
 'этот',
 'ещё',
 'исправить',
 'надо',
 'нужно',
 'тот',
 'функция',
 'долго',
 'код',
 'ни',
 'во

In [16]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [17]:
import numpy as np
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [18]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in X_train], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in X_test], dtype=np.int32)

In [19]:
x_train

array([[  0,   0,   0, ...,   0,   0, 267],
       [  0,   0,   0, ...,   8, 998,  20],
       [  0,   0,   0, ...,  19,  53, 480],
       ...,
       [  0,   0,   0, ...,   0,   0,   5],
       [  0,   0,   0, ...,   0,   0,  22],
       [  0,   0,   0, ...,   0,   2,  19]], dtype=int32)

# Keras model

In [20]:
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping  

In [21]:
num_classes = 2
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_test, num_classes)

In [22]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [23]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [24]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
28/28 [==============================] - 2s 86ms/step - loss: 0.4832 - accuracy: 0.8266 - val_loss: 0.3888 - val_accuracy: 0.8437
Epoch 2/20
28/28 [==============================] - 0s 14ms/step - loss: 0.2911 - accuracy: 0.8788 - val_loss: 0.2149 - val_accuracy: 0.9013
Epoch 3/20
28/28 [==============================] - 0s 15ms/step - loss: 0.1851 - accuracy: 0.9224 - val_loss: 0.1781 - val_accuracy: 0.9215
Epoch 4/20
28/28 [==============================] - 0s 14ms/step - loss: 0.1484 - accuracy: 0.9401 - val_loss: 0.1759 - val_accuracy: 0.9171
Epoch 5/20
28/28 [==============================] - 0s 14ms/step - loss: 0.1286 - accuracy: 0.9518 - val_loss: 0.1757 - val_accuracy: 0.9215
Epoch 6/20
28/28 [==============================] - 0s 15ms/step - loss: 0.1147 - accuracy: 0.9577 - val_loss: 0.1798 - val_accuracy: 0.9234


In [25]:
score = model.evaluate(x_test, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

8/8 [==============================] - 1s 70ms/step - loss: 0.1692 - accuracy: 0.9286


Test score: 0.1691897064447403
Test accuracy: 0.9286075830459595


In [26]:
results = model.predict(x_test, batch_size=batch_size, verbose=1)

8/8 [==============================] - 0s 4ms/step


In [27]:
from sklearn.metrics import roc_auc_score

In [28]:
roc_auc_score(y_test, results[:, 1])

0.9660618414063177

# 2. Предобучаем word2vec и его эмбединга инициализируем сетку, как влияет на качество?

In [29]:
# !pip install gensim

In [30]:
from gensim.models import Word2Vec

In [31]:
sentences = X_train.apply(lambda x: x.split())

In [32]:
modelW2V = Word2Vec(sentences=sentences.values, size=128, window=5, min_count=1)

# Keras model

Модель с trainable=False

In [33]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128,
                    input_length=max_len, weights=[modelW2V.trainables.syn1neg[:max_words]], trainable=False))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [34]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [35]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
28/28 [==============================] - 0s 14ms/step - loss: 0.4771 - accuracy: 0.8214 - val_loss: 0.3793 - val_accuracy: 0.8430
Epoch 2/20
28/28 [==============================] - 0s 10ms/step - loss: 0.3382 - accuracy: 0.8481 - val_loss: 0.3202 - val_accuracy: 0.8544
Epoch 3/20
28/28 [==============================] - 0s 12ms/step - loss: 0.3113 - accuracy: 0.8574 - val_loss: 0.3087 - val_accuracy: 0.8519
Epoch 4/20
28/28 [==============================] - 0s 9ms/step - loss: 0.3047 - accuracy: 0.8612 - val_loss: 0.3027 - val_accuracy: 0.8551
Epoch 5/20
28/28 [==============================] - 0s 9ms/step - loss: 0.2993 - accuracy: 0.8650 - val_loss: 0.2977 - val_accuracy: 0.8500
Epoch 6/20
28/28 [==============================] - 0s 9ms/step - loss: 0.2938 - accuracy: 0.8681 - val_loss: 0.2922 - val_accuracy: 0.8513
Epoch 7/20
28/28 [==============================] - 0s 9ms/step - loss: 0.2872 - accuracy: 0.8715 - val_loss: 0.2859 - val_accuracy: 0.8576
Epoch 8/20
28/28 

In [36]:
results = model.predict(x_test, batch_size=batch_size, verbose=1)

8/8 [==============================] - 0s 6ms/step


In [37]:
roc_auc_score(y_test, results[:, 1])

0.9105290985682343

Модель с обучаемыми весами

In [38]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128,
                    input_length=max_len, weights=[modelW2V.trainables.syn1neg[:max_words]]))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [39]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [40]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
28/28 [==============================] - 1s 20ms/step - loss: 0.3730 - accuracy: 0.8465 - val_loss: 0.2994 - val_accuracy: 0.8430
Epoch 2/20
28/28 [==============================] - 0s 15ms/step - loss: 0.2737 - accuracy: 0.8727 - val_loss: 0.2493 - val_accuracy: 0.8854
Epoch 3/20
28/28 [==============================] - 0s 15ms/step - loss: 0.2218 - accuracy: 0.9089 - val_loss: 0.1991 - val_accuracy: 0.9089
Epoch 4/20
28/28 [==============================] - 0s 14ms/step - loss: 0.1737 - accuracy: 0.9308 - val_loss: 0.1694 - val_accuracy: 0.9158
Epoch 5/20
28/28 [==============================] - 0s 15ms/step - loss: 0.1435 - accuracy: 0.9442 - val_loss: 0.1649 - val_accuracy: 0.9203
Epoch 6/20
28/28 [==============================] - 0s 15ms/step - loss: 0.1229 - accuracy: 0.9541 - val_loss: 0.1714 - val_accuracy: 0.9203


In [41]:
results = model.predict(x_test, batch_size=batch_size, verbose=1)

8/8 [==============================] - 0s 6ms/step


In [42]:
roc_auc_score(y_test, results[:, 1])

0.9685194381647705

Качество этой модели выше чем первой